# Frist Draft Classifier CODE

I changed few things compared to the explained code :

- I put the hyperparameters at the begining of the code
- I put the process of the Test set before fitting the part to have an estimation of each model on the block where I fit the model (to have a better visualisation of the result)
- I changed Size_Data to min(Size_Data , len(Data)) in the different functions. That way, if you set Size data to a very high value, the algorithm use all the data available
- I use np.average(y_predRF == y_test) to get the prediction of a model
- I removed the scaling part that is not used
- I Create simultanetly X_train_tfidf and X_train_cv to use both on different models


## I Import Library

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


## II Hyperparameters

In [2]:
Size_Data = 10000 # Number of sample considerate for training and test
Max_NB_Word = 80000 # Only considerate the 80 000 more common words (110 000 actually)
r_split=0.2

## III Import Data

In [3]:
#Read Data
dataset_Healthy = pd.read_excel(r'C:\Users\Alu\Desktop\S and P Global\Funds Articles_Healthy_New.xlsx', dtype={'Name':str}, quoting = 3)
dataset_Unhealthy = pd.read_excel(r'C:\Users\Alu\Desktop\S and P Global\Fund Articles_Unhealthy_New.xlsx', dtype={'Name':str}, quoting = 3)

#Rename Columns
dataset_Healthy.columns = ['Articles', 'Labels']
dataset_Unhealthy.columns = ['Articles', 'Labels']

#Create Labels
dataset_Healthy['Labels']=1
dataset_Unhealthy['Labels']=0

#Schuffle
dataset_Healthy = dataset_Healthy.sample(frac=1).reset_index(drop=True)
dataset_Unhealthy = dataset_Unhealthy.sample(frac=1).reset_index(drop=True)

#Troncaturate
dataset_Healthy = dataset_Healthy[:min(int(Size_Data/2),len(dataset_Healthy))]
dataset_Unhealthy = dataset_Unhealthy[:min(int(Size_Data/2),len(dataset_Unhealthy))]

#Concatenate
Data = pd.concat([dataset_Healthy,dataset_Unhealthy])
Data = Data.sample(frac=1).reset_index(drop=True)

#Split train and test
Data_train =  Data[:int(Size_Data*(1-r_split))].reset_index(drop=True)
Data_test =  Data[int(Size_Data*(1-r_split)):Size_Data].reset_index(drop=True)


## IV Preprocessing

In [4]:
stopwords = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
def my_tokenizerI(s):
    s = re.sub('[^1-9a-zA-Z]', ' ', s)
    s = s.lower() 
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t) > 2] 
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] 
    tokens = [t for t in tokens if t not in stopwords] 
    s = ' '.join(tokens)
    return s


corpus_train =[]
for i in range (0,int(min(Size_Data,len(Data))*(1-r_split))):
    Tokens = my_tokenizerI(Data_train['Articles'][i])
    corpus_train.append(Tokens)

    
# Creating the Bag of Words model or tfidf model or Glove mean
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
cv = CountVectorizer(max_features=Max_NB_Word)
X_train_cv = cv.fit_transform(corpus_train).toarray()

tfidf=TfidfVectorizer(max_features=Max_NB_Word)
X_train_tfidf=tfidf.fit_transform(corpus_train).toarray()


y_train = Data_train.iloc[:, 1].values

## V Process Test

In [5]:
corpus_test =[]
for i in range (0,int(min(Size_Data,len(Data))*(r_split))):
    Tokens = my_tokenizerI(Data_test['Articles'][i])
    corpus_test.append(Tokens)

X_test_tfidf=tfidf.transform(corpus_test).toarray()
X_test_cv=cv.transform(corpus_test).toarray()

y_test = Data_test.iloc[:, 1].values


## VI Fit Model and evaluate

__Logistic Regression__

In [6]:
from sklearn.linear_model import LogisticRegression
classifierLR = LogisticRegression()
classifierLR.fit(X_train_cv, y_train)
y_predLR=classifierLR.predict(X_test_cv)
np.average(y_predLR == y_test)


0.9065

__Naive Bayes__

In [7]:
from sklearn.naive_bayes import GaussianNB
classifierNB = GaussianNB()
classifierNB.fit(X_train_tfidf, y_train)
y_predNB=classifierNB.predict(X_test_tfidf)
np.average(y_predNB == y_test)

0.8025

__Random Forest__

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifierRF = RandomForestClassifier(n_estimators = 20, criterion = 'entropy')
classifierRF.fit(X_train_tfidf, y_train)
y_predRF=classifierRF.predict(X_test_tfidf)
np.average(y_predRF == y_test)

0.891

__SVM__

In [ ]:
from sklearn.svm import SVC
classifierSVC = SVC(kernel = 'rbf')
classifierSVC.fit(X_train_tfidf, y_train)
y_predSVC=classifierSVC.predict(X_test_tfidf)
np.average(y_predSVC == y_test)